In [5]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

In [ ]:
# create adjacency matrix
feature_df = pd.read_csv('AK_features.csv', header=None)
zero_matrix = np.zeros((len(feature_df), len(feature_df)))
adj_df = pd.DataFrame(zero_matrix)
id_map = {}

for i, id in enumerate(feature_df[0]):
    id_map[id] = i
    
id_map

In [7]:
for node, adj_list in zip(feature_df[0], feature_df[2]):
    node_idx = id_map[node]
    for adj in adj_list.split(','):
        adj_idx = id_map[int(adj[1:])]
        adj_df.iloc[node_idx, adj_idx] = 1
# adj_df
adj_matrix = adj_df.values

In [8]:
adj_matrix

array([[0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
def evaluate(distribution, p_bar):

    # calculate RMSE
    RMSE = ...

    # calculate compactness score
    CS = ...

    return RMSE+CS # misschien normalizen zodat het tussen de 0 en 1 valt voor interpreteerbaarheid?

In [170]:
def create_random_distribution(num_districts, num_units):
    
    keys = [i+1 for i in range(num_districts)]
    distribution = {key: [] for key in keys}
    assigned_units = []

    # randomly initialize district centers, one unit per district
    list_units = list(range(num_units))
    list_districts = list(range(1, num_districts+1))

    for d in range(num_districts):
        random_unit = random.choice(list_units)
        random_district = random.choice(list_districts)
        distribution[random_district].append(random_unit)
        list_units.remove(random_unit)
        list_districts.remove(random_district)
        assigned_units.append(random_unit)
        
    # add other units to districts based on adjacency matrix
    for unit in list_units:

        for i in range(num_units):

            if i in assigned_units and adj_matrix[unit, i] == 1:

                # find key of the unit
                for key, value_list in distribution.items():
                    if i in value_list:
                        found_key = key
                        break
                
                # append the unit to the district where there is also an adjacent unit
                distribution[found_key].append(unit)
                assigned_units.append(unit)
                break

        # in case a unit entered the loop but none of its adjacent units were assigned yet, it is randomly assigned to a district here
        if unit not in assigned_units:
            distribution[np.random.randint(1, num_districts+1)].append(unit)

    len_total = sum(len(lst) for lst in distribution.values())

    return distribution, len_total

In [202]:
create_random_distribution(4, 10)

({1: [2], 2: [9, 7], 3: [0, 1, 3, 4, 6, 8], 4: [5]}, 10)

In [ ]:
# basic framework for simulated annealing step
def SimulatedAnnealing(objective, bounds, n_iterations, step_size, temp, num_districts, num_units):

    # generate initial district distribution
    best_solution = create_random_distribution(num_districts, num_units)

    # evaluate this distribution using evaluate
    best_evaluate = evaluate(best_solution)

    # current working solution
    current_distribution, current_evaluate = best_solution, best_evaluate
    
    for i in range(n_iterations):
        
        # take a step by randomly perturbing previous distribution
        candidate_distribution = perturb()

        # evaluate step using MSE
        candidate_evaluate = evaluate()
    
        # check for new best solution
        if candidate_evaluate < current_evaluate: # for minimization, use > for maximiation problem
            best_solution, best_evaluate = candidate_distribution, candidate_evaluate
            #print('>%d f(%s) = %.5f' % (i, best, best_eval))
    
        # calculate difference in scores
        diff = candidate_evaluate-current_evaluate
    
        # calculate temp for each epoch
        t = temp / float(i + 1)
    
        # calculate metropolis acceptance criterion
        metropolis = math.exp(-diff / t)
    
        # check whether we should keep new point
        if diff < 0 or np.random.rand() < metropolis:
            curr, curr_eval = candidate_distribution, candidate_evaluate

    return [best, best_eval]